## 1 Batch Normalization

In [1]:
from tensorflow import keras

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

### 1-1 BN 的基本用法

In [2]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax'),
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [4]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [5]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['accuracy'])

In [7]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 13s 239us/sample - loss: 0.8560 - accuracy: 0.7131 - val_loss: 0.5546 - val_accuracy: 0.8110
Epoch 2/10
55000/55000 [==============================] - 12s 214us/sample - loss: 0.5790 - accuracy: 0.7987 - val_loss: 0.4782 - val_accuracy: 0.8380
Epoch 3/10
55000/55000 [==============================] - 12s 214us/sample - loss: 0.5201 - accuracy: 0.8184 - val_loss: 0.4416 - val_accuracy: 0.8492
Epoch 4/10
55000/55000 [==============================] - 12s 214us/sample - loss: 0.4836 - accuracy: 0.8305 - val_loss: 0.4198 - val_accuracy: 0.8570
Epoch 5/10
55000/55000 [==============================] - 12s 213us/sample - loss: 0.4577 - accuracy: 0.8388 - val_loss: 0.4036 - val_accuracy: 0.8628
Epoch 6/10
55000/55000 [==============================] - 12s 217us/sample - loss: 0.4397 - accuracy: 0.8441 - val_loss: 0.3934 - val_accuracy: 0.8682
Epoch 7/10
55000/55000 [=====================

### 1-2 在激活函数前使用 BN

* 需要将激活函数与网络层定义分开，且设置use_bias=False

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dense(10, activation='softmax'),
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 13s 234us/sample - loss: 1.0145 - accuracy: 0.6822 - val_loss: 0.6564 - val_accuracy: 0.7954
Epoch 2/10
55000/55000 [==============================] - 12s 214us/sample - loss: 0.6656 - accuracy: 0.7852 - val_loss: 0.5467 - val_accuracy: 0.8232
Epoch 3/10
55000/55000 [==============================] - 12s 212us/sample - loss: 0.5818 - accuracy: 0.8063 - val_loss: 0.4940 - val_accuracy: 0.8386
Epoch 4/10
55000/55000 [==============================] - 12s 211us/sample - loss: 0.5388 - accuracy: 0.8171 - val_loss: 0.4632 - val_accuracy: 0.8460
Epoch 5/10
55000/55000 [==============================] - 11s 209us/sample - loss: 0.5095 - accuracy: 0.8261 - val_loss: 0.4419 - val_accuracy: 0.8524
Epoch 6/10
55000/55000 [==============================] - 12s 213us/sample - loss: 0.4875 - accuracy: 0.8317 - val_loss: 0.4266 - val_accuracy: 0.8548
Epoch 7/10
55000/55000 [=====================

### 1-3 梯度裁剪

* 裁剪梯度的思想是控制梯度不要超过某个阈值。这一技术在 RNN 中常用，因为 RNN 不能设置 BN 层。
* 由 optimizer 的 clipvalue 和 clipnorm 参数控制。需要注意如果修改 clipvalue 可能会改变梯度的方向。所以最好用 clipnorm。

In [11]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

In [12]:
optimizer = keras.optimizers.SGD(clipnorm=1.0)

## 2 迁移学习 - 预训练层重用

### 2-1 在小批量样本上训练

In [14]:
import numpy as np

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [15]:
X_train_A.shape

(43986, 28, 28)

In [16]:
X_train_B.shape

(200, 28, 28)

In [17]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [18]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

#### 在大量样本上进行预训练

In [20]:
import tensorflow as tf

tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [21]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 8s 189us/sample - loss: 0.5909 - accuracy: 0.8100 - val_loss: 0.3772 - val_accuracy: 0.8707
Epoch 2/20
43986/43986 [==============================] - 8s 171us/sample - loss: 0.3520 - accuracy: 0.8793 - val_loss: 0.3400 - val_accuracy: 0.8772
Epoch 3/20
43986/43986 [==============================] - 8s 172us/sample - loss: 0.3167 - accuracy: 0.8890 - val_loss: 0.3018 - val_accuracy: 0.8954
Epoch 4/20
43986/43986 [==============================] - 7s 170us/sample - loss: 0.2969 - accuracy: 0.8969 - val_loss: 0.2859 - val_accuracy: 0.9028
Epoch 5/20
43986/43986 [==============================] - 7s 168us/sample - loss: 0.2824 - accuracy: 0.9030 - val_loss: 0.2808 - val_accuracy: 0.9066
Epoch 6/20
43986/43986 [==============================] - 8s 172us/sample - loss: 0.2721 - accuracy: 0.9073 - val_loss: 0.2684 - val_accuracy: 0.9103
Epoch 7/20
43986/43986 [===========================

In [22]:
model_A.save("./models/Keras6-modelA.h5")

#### 直接在 B 样本上训练试试

In [23]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 1s 5ms/sample - loss: 0.9545 - accuracy: 0.4600 - val_loss: 0.6655 - val_accuracy: 0.5385
Epoch 2/20
200/200 [==============================] - 0s 821us/sample - loss: 0.5899 - accuracy: 0.6900 - val_loss: 0.4785 - val_accuracy: 0.8519
Epoch 3/20
200/200 [==============================] - 0s 798us/sample - loss: 0.4512 - accuracy: 0.8800 - val_loss: 0.4098 - val_accuracy: 0.8945
Epoch 4/20
200/200 [==============================] - 0s 825us/sample - loss: 0.3871 - accuracy: 0.9100 - val_loss: 0.3666 - val_accuracy: 0.9128
Epoch 5/20
200/200 [==============================] - 0s 788us/sample - loss: 0.3438 - accuracy: 0.9250 - val_loss: 0.3315 - val_accuracy: 0.9300
Epoch 6/20
200/200 [==============================] - 0s 796us/sample - loss: 0.3095 - accuracy: 0.9300 - val_loss: 0.3034 - val_accuracy: 0.9402
Epoch 7/20
200/200 [==============================] - 0s 795us/sample - loss: 0.

#### 在 B 样本集上微调 A 的预训练参数

In [25]:
model_A = keras.models.load_model("./models/Keras6-modelA.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # 用 A 的网络层实现参数共享
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [26]:
model_A_clone = keras.models.clone_model(model_A) # 为了不影响 A 的参数，对 A 进行克隆
model_A_clone.set_weights(model_A.get_weights())

In [27]:
for layer in model_B_on_A.layers[:-1]: # 前几个轮次冻结重用层
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [28]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]: # 解冻
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),  # 可以考虑降低学习率
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 4ms/sample - loss: 0.2787 - accuracy: 0.9250 - val_loss: 0.2925 - val_accuracy: 0.9229
Epoch 2/4
200/200 [==============================] - 0s 698us/sample - loss: 0.2678 - accuracy: 0.9300 - val_loss: 0.2823 - val_accuracy: 0.9290
Epoch 3/4
200/200 [==============================] - 0s 696us/sample - loss: 0.2578 - accuracy: 0.9350 - val_loss: 0.2728 - val_accuracy: 0.9310
Epoch 4/4
200/200 [==============================] - 0s 709us/sample - loss: 0.2486 - accuracy: 0.9400 - val_loss: 0.2642 - val_accuracy: 0.9351
Train on 200 samples, validate on 986 samples
Epoch 1/16
200/200 [==============================] - 1s 5ms/sample - loss: 0.2224 - accuracy: 0.9450 - val_loss: 0.2115 - val_accuracy: 0.9604
Epoch 2/16
200/200 [==============================] - 0s 773us/sample - loss: 0.1754 - accuracy: 0.9550 - val_loss: 0.1771 - val_accuracy: 0.9726
Epoch 3/16
200/200 [====================

In [30]:
model_B.evaluate(X_test_B, y_test_B, verbose=0)

[0.14446662282943726, 0.9695]

In [34]:
model_B_on_A.evaluate(X_test_B, y_test_B, verbose=0) # 准确率有效提高

[0.056635786026716234, 0.994]